# File B : Compute IV
## This file has now been converted to a python class  (interpol.py in src)
<br>
This file operates on only one year of data as elaboration.

In [1]:
import sys
sys.path.insert(0, "/Users/shah/CODE_BOOK_3/code_document/MyRepo/THESIS")
sys.modules.pop("src", None)

In [2]:

import pandas as pd
import numpy as np
from math import sqrt, pi, exp
from arch import arch_model
import yfinance as yf 
from scipy.optimize import minimize
import warnings
from arch.__future__ import reindexing
from arch.utility.exceptions import ConvergenceWarning
from src.msGarch import msGARCH
from src.msGarch import msGARCHIV
from tqdm import tqdm
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message="y is poorly scaled")
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from src.metrics import metrics

## Load Data

In [5]:
# Hourly data
d_btc = pd.read_parquet("../../data/rawData/btc_underlying_minutely_2025.parquet")


In [6]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../../data/rawData")
YEARS = [2022, 2023, 2024, 2025]

d_btc_raw_by_year = {}
df_raw_by_year = {}

for y in YEARS:
    d_btc_raw_by_year[y] = pd.read_parquet(DATA_DIR / f"btc_underlying_minutely_{y}.parquet")
    df_raw_by_year[y]    = pd.read_parquet(DATA_DIR / f"derivative_data_{y}.parquet")


## Using Current MFIV

In [9]:
from src.interpol import MFIVPipeline

In [10]:
file_path0 = "../../data/rawData/d_2025.parquet" # contains stock price. Uniform Index
file_path1 = "../../data/rawData/df_2025.parquet" # # contains order book. Non-Uniform Index
d_btc = pd.read_parquet(file_path0, engine="pyarrow")
df    = pd.read_parquet(file_path1, engine="pyarrow")



In [12]:
from src.interpol import MFIVPipeline
import numpy as np

q = 0.0
HOURS = [5, 20, 30, 40]

for H in HOURS:
    T = H / (24.0 * 365.0)  # years
    d_btc[f"F{H}"] = d_btc["Close"] * np.exp((d_btc["rfr"] - q) * T)


In [13]:
# df already has DatetimeIndex and columns ["Close","rt"]
idx = d_btc.index.tz_localize("UTC") if d_btc.index.tz is None else d_btc.index.tz_convert("UTC")
rt  = d_btc["r"].reindex(idx)

# compute hourly MFIV/BSIV on the same hours as idx
pipe = MFIVPipeline(df=df, d_btc=d_btc, q=0.0)
pipe.build_surfaces()
MFIV_byH = pipe.compute_mfiv_byH(H_LIST=[5])     # H must be defined (e.g., 20)

mf = MFIV_byH[5].reindex(idx.tz_convert(None))   # MFIV_byH index is tz-naive
bsiv = mf["bsIV"].to_numpy()
mfiv = mf["MFIV"].to_numpy()


X = pd.DataFrame({"rt": rt.values, "bsIV": bsiv, "MFIV": mfiv}, index=idx)

100%|██████████| 8782/8782 [00:04<00:00, 1847.24it/s]


In [14]:
X.isna().sum()

rt        1
bsIV    484
MFIV    484
dtype: int64

In [16]:
# Save once
file_path = ("../../data/readyData/d_X_OneYear.parquet")
X.to_parquet(file_path)

## or X with fwd MFIV from t-H (For Model T-F) 

In [ ]:
from src.interpol import MFIVPipeline
import pandas as pd
import numpy as np

H = 5
H_LIST = [5, 20, 30, 40]
q = 0.0

pipe = MFIVPipeline(df=df, d_btc=d_btc, q=q)
df11, df12, MFIV_byH = pipe.run(H_LIST)

d_btc["r"] = np.log(d_btc["Close"]).diff()

idx = d_btc.index.tz_localize("UTC") if d_btc.index.tz is None else d_btc.index.tz_convert("UTC")
rt  = d_btc["r"].reindex(idx)

pipe = MFIVPipeline(df=df, d_btc=d_btc, q=0.0)
pipe.build_surfaces()
MFIV_byH = pipe.compute_mfiv_byH(H_LIST=[H])

# --- SHIFT: move MFIV computed at t to be indexed at t+H hours ---
mf_src = MFIV_byH[H].copy()                      # tz-naive index
mf_src.index = mf_src.index + pd.Timedelta(hours=0)

# align to your returns index (tz-naive for reindex)
mf = mf_src.reindex(idx.tz_convert(None))

bsiv = mf["bsIV"].to_numpy()
mfiv = mf["MFIV"].to_numpy()

xB = pd.DataFrame({"rt": rt.values, "bsIV": bsiv, "MFIV": mfiv}, index=idx)


In [ ]:
xB.isna().sum()

## Load Data

In [ ]:
file_path = ("../data/d_X_OneYear.parquet")
X = pd.read_parquet(file_path)

In [ ]:
eps = 1e-12
X[["bsIV","MFIV"]] = np.exp(
    np.log(X[["bsIV","MFIV"]].clip(lower=eps))
      .interpolate(method="time", limit_direction="both")
)
X